In [136]:
import os
import pandas as pd
import numpy as np
import copy

In [137]:
cn_dict = np.load("data/cn_dict.npy",allow_pickle='TRUE').item()
ad_dict = np.load("data/ad_dict.npy",allow_pickle='TRUE').item()
smci_dict = np.load("data/smci_dict.npy",allow_pickle='TRUE').item()
pmci_dict = np.load("data/pmci_dict.npy",allow_pickle='TRUE').item()

In [138]:
IMAGES = pd.read_csv("data/Big_cohort_MRI_Images_25Oct2024.csv")
IMAGES

,image_id,subject_id,mri_visit,mri_date,mri_description,mri_type,mri_weighting,mri_sequence,mri_thickness,mri_te,...,mri_flip_angle,mri_acq_plane,mri_width,mri_height,mri_n_images,mri_pixel_x,mri_pixel_y,mri_mfr,mri_mfr_model,mri_field_str
0,70846,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
1,70847,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
2,70848,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,SAGITTAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
3,70849,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,AXIAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
4,70850,037_S_1421,sc,2007-08-27,"Head,3-P,2D,LOCALIZER",2D,T2,RM,5.0,1.600000,...,30,CORONAL,256.0,256.0,5,1.01563,1.01563,GE MEDICAL SYSTEMS,GENESIS_SIGNA,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115989,10967970,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,18.440001,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115990,10967971,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,10.620000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115991,10967972,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,6.710000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0
115992,10967973,011_S_10261,4_sc,2024-10-02,Axial 3D ME T2 GRE (MSV21),3D,T2,GR,1.8,22.350000,...,15,AXIAL,384.0,384.0,88,0.52083,0.52083,SIEMENS,Prisma_fit,3.0


**Dict where patient id's are keys and the values are every single image with correspinding visit code.**

In [139]:
image_dict = {}

for index, row in IMAGES.iterrows():

    #Add filters to decide what images to include.
    if row['mri_type'] == '3D':

        if row['subject_id'] in image_dict:
            image_dict[row['subject_id']].append([row['mri_visit'], row['image_id']])
        else:
            image_dict[row['subject_id']] = [[row['mri_visit'], row['image_id']]]

In [140]:
image_dict['016_S_1326']

[['sc', 42540],
 ['sc', 42541],
 ['sc', 42544],
 ['sc', 42545],
 ['bl', 48259],
 ['bl', 48260],
 ['bl', 48262],
 ['bl', 48263],
 ['m06', 76283],
 ['m06', 76284],
 ['m06', 76287],
 ['m06', 76288],
 ['m06', 76817],
 ['m06', 76818],
 ['m06', 76820],
 ['m06', 76821],
 ['m12', 99751],
 ['m12', 99752],
 ['m12', 99755],
 ['m12', 99756],
 ['m12', 100326],
 ['m12', 100327],
 ['m12', 100329],
 ['m12', 100330],
 ['m18', 119312],
 ['m18', 119313],
 ['m18', 119316],
 ['m18', 119317],
 ['m18', 122160],
 ['m18', 122161],
 ['m18', 122163],
 ['m18', 122164],
 ['m24', 140315],
 ['m24', 140316],
 ['m24', 140319],
 ['m24', 140320],
 ['m24', 141837],
 ['m24', 141838],
 ['m24', 141840],
 ['m24', 141841],
 ['m36', 173998],
 ['m36', 174002],
 ['m36', 174005],
 ['m36', 174006],
 ['m48', 233787],
 ['m48', 233790],
 ['m48', 233792],
 ['m48', 233793],
 ['v06', 310806],
 ['v06', 310807],
 ['v06', 310808],
 ['v06', 310809],
 ['v11', 363775],
 ['v11', 363776],
 ['v11', 363777],
 ['v11', 363780],
 ['v11', 363781]]

**Function for combining patient dict with image dict**

In [141]:
def combine_patients_pictures(in_dict):  

    dict_with_images = copy.deepcopy(in_dict)

    for patient in dict_with_images:

        if patient in image_dict:
            visits_pictures = image_dict[patient]

            for visit in dict_with_images[patient]:
                visit.append(list())

                for picture in visits_pictures: 

                    if visit[0] == picture[0]:
                        visit[3].append(picture[1])
        else:
            pass

    return dict_with_images


In [142]:
cn_dict_with_images = combine_patients_pictures(cn_dict)
ad_dict_with_images = combine_patients_pictures(ad_dict)
smci_dict_with_images = combine_patients_pictures(smci_dict)
pmci_dict_with_images = combine_patients_pictures(pmci_dict)


print(cn_dict_with_images['011_S_0005'])
print(ad_dict_with_images['011_S_0003'])
print(smci_dict_with_images['022_S_0004'])
print(pmci_dict_with_images['023_S_0030'])

[['bl', '2005-09-30', 1.0, []], ['m06', '2006-03-09', 1.0, [11735, 11736, 11738, 11739]], ['m12', '2006-09-06', 1.0, [24618, 24617, 24615, 24614]], ['m24', '2007-09-07', 1.0, [193493, 193495, 193496, 193500]], ['m36', '2008-09-10', 1.0, [117773, 117774, 117776, 117777]]]
[['bl', '2005-09-30', 3.0, []], ['m06', '2006-03-13', 3.0, [16544, 16545, 16547, 16548]], ['m12', '2006-09-13', 3.0, [24693, 24694, 24696, 24697]], ['m24', '2007-09-12', 3.0, [73246, 73247, 73249, 73250]]]
[['bl', '2005-11-08', 2.0, []], ['m06', '2006-05-02', 2.0, [16098, 16099, 16101, 16102]], ['m12', '2006-11-14', 2.0, [30796, 30797, 30799, 30800]], ['m18', '2007-05-14', 2.0, [65889, 65890, 65892, 65893]], ['m36', '2008-11-18', 2.0, [130023, 130024, 130026, 130027]]]
[['bl', '2005-10-20', 2.0, [8249, 8248, 8246, 8245]]]


**Checking for any patient overlap between datasets**

In [143]:
if set(cn_dict_with_images).intersection(ad_dict_with_images) or set(cn_dict_with_images).intersection(smci_dict_with_images) or set(cn_dict_with_images).intersection(pmci_dict_with_images) or set(ad_dict_with_images).intersection(smci_dict_with_images) or set(ad_dict_with_images).intersection(pmci_dict_with_images) or set(smci_dict_with_images).intersection(pmci_dict_with_images):
    print("Error. Patient overlap")

else:
    print("No patient overlap.")

No patient overlap.
